#Setup

In [ ]:
!pip install eurostat

In [ ]:
import requests
import pandas as pd
import eurostat
import json
import os

In [ ]:
#Trick to get TOC (json or excel as well
toc_df = eurostat.get_toc_df(agency='all', dataset='all', lang='en')
toc_df.to_json('eurostat_catalog.json', orient='records', indent=4)

# LOOKUP by data names from the uploaed eurostat_catalog.json

In [ ]:
# Load the catalog JSON file
with open('eurostat_catalog.json', 'r') as f:
    catalog = json.load(f)

In [ ]:
# Search for a dataset (example: "farm")
search_term = "farm"
search_results = [item for item in catalog if search_term.lower() in item['title'].lower()]

In [ ]:
# Display the first few results
print(json.dumps(search_results[:150], indent=4, ensure_ascii=False))
# copy the needed "code" and use it further in the fetch data module

[
    {
        "title": "Annual production and utilisation of milk on the farm",
        "code": "MED_AG32",
        "type": "dataset",
        "last update of data": "2016-08-29T23:00:00+0200",
        "last table structure change": "2024-01-10T23:00:00+0100",
        "data start": "2000",
        "data end": "2015"
    },
    {
        "title": "Poultry farming",
        "code": "MED_AG34",
        "type": "dataset",
        "last update of data": "2020-05-28T23:00:00+0200",
        "last table structure change": "2024-01-10T23:00:00+0100",
        "data start": "2005",
        "data end": "2018"
    },
    {
        "title": "Production of cow's milk on farms by NUTS 2 region",
        "code": "TGS00046",
        "type": "dataset",
        "last update of data": "2024-10-07T23:00:00+0200",
        "last table structure change": "2024-10-07T23:00:00+0200",
        "data start": "2012",
        "data end": "2023"
    },
    {
        "title": "Farm labour force",
        "code": "TAG

# Fetch Data



[Euriostat rest api data](https://ec.europa.eu/eurostat/web/user-guides/data-browser/api-data-access/api-getting-started#fragment-15944082-uabj-inline-nav-3)

##  API list:

*   Statistics API: Offers data in JSON-stat 2.0 format, optimized for visualization tools, supporting REST protocol and CORS; [webpage](https://ec.europa.eu/eurostat/web/user-guides/data-browser/api-data-access/api-getting-started/api?utm_source=chatgpt.com)
  - Placeholder data: https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/DATASET_CODE?lang=EN

*   SDMX 3.0 API: Enables programmatic access to metadata and data following the SDMX 3.0 specification, multlp formats , get comlete dataset;
* SDMX 2.1 API: Provides access to metadata and data adhering to the SDMX 2.1 specification, supporting various data formats for customized queries;
* Catalogue API: Facilitates dataset discovery through programmatic access to Eurostat's catalogue of datasets, available in XML and DCAT-AP formats (SPARQL endpoint at https://data.europa.eu/sparql)

In [ ]:
#selected: "title": "Share of family farms by NUTS 2 region","code": "EF_LF_FAM_SH" .
# or select : EF_M_FARMANG in "title": "Farm indicators by age and sex of the manager, economic size of the farm, utilised agricultural area and NUTS 2 region",

In [ ]:
# Define the dataset code
dataset_code = "EF_M_FARMANG" # the one in the cell of selected: ..."code"

# Construct the API URL for the dataset
url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{dataset_code}?format=JSON&lang=EN'

# Fetch the data
response = requests.get(url)

# Check if the response is successful
if response.status_code == 200:
    data = response.json()

    # Save the JSON to a file
    with open(f'{dataset_code}_metadata.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f" saved to {dataset_code}_metadata.json")
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")


 saved to EF_M_FARMANG_metadata.json


# Exploring keys used and, then, explicative docs in metadata, go to next section to get some documents to study how to use the dataset

In [ ]:
# Load the JSON file
file_path = f'{dataset_code}_metadata.json'
with open(file_path, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# Flatten the JSON
def flatten_json(data, parent_key='', sep='.'):
    """Flatten a nested JSON object into a flat dictionary."""
    flattened = {}
    if isinstance(data, dict):
        for key, value in data.items():
            full_key = f"{parent_key}{sep}{key}" if parent_key else key
            flattened.update(flatten_json(value, full_key, sep=sep))
    elif isinstance(data, list):
        for i, item in enumerate(data):
            full_key = f"{parent_key}{sep}{i}" if parent_key else str(i)
            flattened.update(flatten_json(item, full_key, sep=sep))
    else:
        flattened[parent_key] = data
    return flattened

flattened_metadata = flatten_json(metadata)

# Convert the flattened metadata to a DataFrame
df = pd.DataFrame(flattened_metadata.items(), columns=['Key', 'Value'])

# Inspect the first few rows
print(df.tail(20))


                                   Key  \
9804700    extension.annotation.6.type   
9804701   extension.annotation.6.title   
9804702    extension.annotation.6.href   
9804703    extension.annotation.7.type   
9804704   extension.annotation.7.title   
9804705    extension.annotation.7.href   
9804706    extension.annotation.8.type   
9804707   extension.annotation.8.title   
9804708    extension.annotation.9.type   
9804709   extension.annotation.9.title   
9804710   extension.annotation.10.type   
9804711  extension.annotation.10.title   
9804712   extension.annotation.11.type   
9804713   extension.annotation.11.text   
9804714   extension.annotation.12.type   
9804715   extension.annotation.12.date   
9804716   extension.annotation.13.type   
9804717   extension.annotation.13.date   
9804718       extension.status.label.c   
9804719       extension.status.label.u   

                                                     Value  
9804700                                          ESMS_HT

In [ ]:
df.tail(100)
# click on interactive table visualization, filter if needed

,Key,Value
9804620,dimension.geo.category.label.UKC2,Northumberland and Tyne and Wear (NUTS 2021)
9804621,dimension.geo.category.label.UKD1,Cumbria (NUTS 2021)
9804622,dimension.geo.category.label.UKD3,Greater Manchester (NUTS 2021)
9804623,dimension.geo.category.label.UKD4,Lancashire (NUTS 2021)
9804624,dimension.geo.category.label.UKD6,Cheshire (NUTS 2021)
...,...,...
9804715,extension.annotation.12.date,2024-03-26T23:00:00+0100
9804716,extension.annotation.13.type,UPDATE_STRUCTURE
9804717,extension.annotation.13.date,2024-01-10T11:00:00+0100
9804718,extension.status.label.c,confidential


In [ ]:
# Check for 'dimension' key to explore dimensions and their categories
if 'dimension' in metadata:
    dimensions = metadata['dimension']
    print("\nDimensions available in the JSON:")
    for dim_name, dim_data in dimensions.items():
        print(f"Dimension: {dim_name}")
        if 'label' in dim_data:
            print(f"  Description: {dim_data['label']}")
        if 'category' in dim_data:
            print("  Categories:")
            categories = dim_data['category']
            # Extract category indices and labels
            index = categories.get('index', {})
            labels = categories.get('label', {})
            for code, idx in index.items():
                description = labels.get(code, 'No description available')
                print(f"    Code: {code} -> {description}")
else:
    print("\nNo 'dimension' key found in the JSON.")



Dimensions available in the JSON:
Dimension: freq
  Description: Time frequency
  Categories:
    Code: A -> Annual
Dimension: sex
  Description: Sex
  Categories:
    Code: T -> Total
    Code: M -> Males
    Code: F -> Females
Dimension: age
  Description: Age class
  Categories:
    Code: TOTAL -> Total
    Code: Y_LT25 -> Less than 25 years
    Code: Y25-34 -> From 25 to 34 years
    Code: Y35-39 -> From 35 to 39 years
    Code: Y35-44 -> From 35 to 44 years
    Code: Y40-44 -> From 40 to 44 years
    Code: Y45-54 -> From 45 to 54 years
    Code: Y55-64 -> From 55 to 64 years
    Code: Y_GE65 -> 65 years or over
Dimension: so_eur
  Description: Standard output in Euros
  Categories:
    Code: TOTAL -> Total
    Code: KE0 -> Zero euros
    Code: KE_GT0_LT2 -> Over zero euros to less than 2000 euros
    Code: KE2-3 -> From 2 000 to 3 999 euros
    Code: KE4-7 -> From 4 000 to 7 999 euros
    Code: KE8-14 -> From 8 000 to 14 999 euros
    Code: KE15-24 -> From 15 000 to 24 999 euros


In [ ]:
# Filter rows where the Key contains "href" (URLs)
urls = df[df['Key'].str.contains('href')]

# Extract relevant columns: Key (description) and Value (URL)
urls_filtered = urls[['Key', 'Value']]

# Convert to dictionary for JSON export
urls_dict = urls_filtered.set_index('Key')['Value'].to_dict()

# Save to a JSON file
output_file = 'extracted_urls.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(urls_dict, f, ensure_ascii=False, indent=4)

print(f"URLs saved to {output_file}")
#From the last output above : extracted_url.json
print(json.dumps(urls_dict, indent=1, ensure_ascii=False))

URLs saved to extracted_urls.json
{
 "extension.annotation.2.href": "https://ec.europa.eu/eurostat/databrowser-backend/api/public/explanatory-notes/get/Info_note_EF_M_FARMANG_20230427.pdf",
 "extension.annotation.3.href": "https://ec.europa.eu/eurostat/databrowser-backend/api/public/explanatory-notes/get/Info_note_EF_M_FARMANG_20230427.pdf",
 "extension.annotation.4.href": "https://ec.europa.eu/eurostat/databrowser-backend/api/public/explanatory-notes/get/Info_note_EF_M_FARMANG_20230427.pdf",
 "extension.annotation.6.href": "https://ec.europa.eu/eurostat/cache/metadata/en/ef_sims.htm",
 "extension.annotation.7.href": "https://ec.europa.eu/eurostat/api/dissemination/files?file=metadata/ef_sims.sdmx.zip"
}


In [ ]:
# open the htm link

# Exploration of dataset var part II

In [88]:
data['dimension']['age']['category']['index']

{'TOTAL': 0,
 'Y_LT25': 1,
 'Y25-34': 2,
 'Y35-39': 3,
 'Y35-44': 4,
 'Y40-44': 5,
 'Y45-54': 6,
 'Y55-64': 7,
 'Y_GE65': 8}

In [82]:
print(data.keys())

dict_keys(['version', 'class', 'label', 'source', 'updated', 'value', 'status', 'id', 'size', 'dimension', 'extension'])


In [83]:
print(data['dimension'].keys())

dict_keys(['freq', 'sex', 'age', 'so_eur', 'uaarea', 'crops', 'unit', 'geo', 'time'])


In [84]:
print(data['dimension']['age']['category'])
print(data['dimension']['sex']['category'])

{'index': {'TOTAL': 0, 'Y_LT25': 1, 'Y25-34': 2, 'Y35-39': 3, 'Y35-44': 4, 'Y40-44': 5, 'Y45-54': 6, 'Y55-64': 7, 'Y_GE65': 8}, 'label': {'TOTAL': 'Total', 'Y_LT25': 'Less than 25 years', 'Y25-34': 'From 25 to 34 years', 'Y35-39': 'From 35 to 39 years', 'Y35-44': 'From 35 to 44 years', 'Y40-44': 'From 40 to 44 years', 'Y45-54': 'From 45 to 54 years', 'Y55-64': 'From 55 to 64 years', 'Y_GE65': '65 years or over'}}
{'index': {'T': 0, 'M': 1, 'F': 2}, 'label': {'T': 'Total', 'M': 'Males', 'F': 'Females'}}


In [89]:
print(data['dimension']['geo']['category'])

{'index': {'EU': 0, 'EU27_2020': 1, 'EU28': 2, 'EU27_2007': 3, 'BE': 4, 'BE10': 5, 'BE21': 6, 'BE22': 7, 'BE23': 8, 'BE24': 9, 'BE25': 10, 'BE31': 11, 'BE32': 12, 'BE33': 13, 'BE34': 14, 'BE35': 15, 'BG': 16, 'BG31': 17, 'BG32': 18, 'BG33': 19, 'BG34': 20, 'BG41': 21, 'BG42': 22, 'CZ': 23, 'CZ01': 24, 'CZ02': 25, 'CZ03': 26, 'CZ04': 27, 'CZ05': 28, 'CZ06': 29, 'CZ07': 30, 'CZ08': 31, 'DK': 32, 'DK01': 33, 'DK02': 34, 'DK03': 35, 'DK04': 36, 'DK05': 37, 'DE': 38, 'DE11': 39, 'DE12': 40, 'DE13': 41, 'DE14': 42, 'DE21': 43, 'DE22': 44, 'DE23': 45, 'DE24': 46, 'DE25': 47, 'DE26': 48, 'DE27': 49, 'DE30': 50, 'DE40': 51, 'DE50': 52, 'DE60': 53, 'DE71': 54, 'DE72': 55, 'DE73': 56, 'DE80': 57, 'DE91': 58, 'DE92': 59, 'DE93': 60, 'DE94': 61, 'DEA1': 62, 'DEA2': 63, 'DEA3': 64, 'DEA4': 65, 'DEA5': 66, 'DEB1': 67, 'DEB2': 68, 'DEB3': 69, 'DEC0': 70, 'DED2': 71, 'DED4': 72, 'DED5': 73, 'DEE0': 74, 'DEF0': 75, 'DEG0': 76, 'EE': 77, 'EE00': 78, 'IE': 79, 'IE04': 80, 'IE05': 81, 'IE06': 82, 'IE01': 8

In [91]:
print(data['id'])

['freq', 'sex', 'age', 'so_eur', 'uaarea', 'crops', 'unit', 'geo', 'time']


In [92]:
print(data['size'])

[1, 3, 9, 12, 10, 1, 5, 396, 4]


In [93]:
dim_sizes = data['size']
freq_size, sex_size, age_size, so_eur_size, uaarea_size, crops_size, unit_size, geo_size, time_size = dim_sizes